In [70]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

import joblib

In [71]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_18488\126190450.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_18488\126190450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_18488\126190450.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_18488\126190450.py:4: FutureWarning: YF.download() has changed argument auto_adjus

In [72]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400
3,2020-01-03,71.83328247070312,72.59404771603535,71.6086769224392,71.76565905712691,146322800
4,2020-01-06,72.40567779541016,72.44432080433776,70.7030121336898,70.95418800651902,118387200


In [73]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100
3,2020-01-03,151.13662719726562,152.40388225400224,150.60304918156822,150.85079221664952,21116200
4,2020-01-06,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700


In [74]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [75]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [76]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400,2020-01-02
3,71.83328247070312,72.59404771603535,71.6086769224392,71.76565905712691,146322800,2020-01-03
4,72.40567779541016,72.44432080433776,70.7030121336898,70.95418800651902,118387200,2020-01-06
5,72.06514739990234,72.67134068922641,71.84536944211234,72.41533721540509,108872000,2020-01-07
6,73.22441864013672,73.52631026770732,71.76809372355218,71.76809372355218,132079200,2020-01-08


In [77]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100,2020-01-02
3,151.13662719726562,152.40388225400224,150.60304918156822,150.85079221664952,21116200,2020-01-03
4,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700,2020-01-06
5,150.1457061767578,152.13710084875345,149.89797766820752,151.8036218417332,21634100,2020-01-07
6,152.53732299804688,153.21383324803998,150.498283797712,151.43204560824276,27746500,2020-01-08


In [78]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [79]:
df_nvidia.head()

,Close,High,Low,Open,Volume,Date
2,5.971746921539307,5.971746921539307,5.892343086871904,5.942872843002966,237536000,2020-01-02
3,5.876162528991699,5.919971911233805,5.827125931377221,5.852017636623186,205384000,2020-01-03
4,5.9008049964904785,5.9060322118773705,5.756682929526284,5.782819006460745,262636000,2020-01-06
5,5.972245216369629,6.018045791935149,5.884128876158687,5.929182638702775,314856000,2020-01-07
6,5.98344612121582,6.024766291068211,5.927937929074284,5.968013252371926,277108000,2020-01-08


In [80]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 2 to 1411
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1410 non-null   object
 1   High    1410 non-null   object
 2   Low     1410 non-null   object
 3   Open    1410 non-null   object
 4   Volume  1410 non-null   object
 5   Date    1410 non-null   object
dtypes: object(6)
memory usage: 66.2+ KB


In [81]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1410,1410,1410,1410,1410,1410
unique,1394,1410,1410,1410,1408,1410
top,139.26516723632812,230.8000030517578,227.07000732421875,228.00999450683594,90956700,2025-08-12
freq,2,1,1,1,2,1


In [82]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [83]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']
X_train, X_test, y_train, y_test = None, None, None, None

In [84]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])

In [85]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year
    
    return df

In [86]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [87]:
#PRINTING SEABORN PLOTS
def print_plots(df, stock):
    for year in years:
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='RSI', kind='line')
        plt.title(f'RSI trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()
        
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
        plt.title(f'MACD trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()

In [88]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
def create_pipeline():
    model = clone(Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
        ]))
    return model

In [90]:
def evaluate(model):
    kfold = KFold(n_splits=6, random_state=30, shuffle=True)

    params = {'lr__n_jobs': [10,20,50]}
    cv = clone(GridSearchCV(estimator=model,param_grid=params,cv=kfold))

    cv.fit(X_train, y_train)
    
    return {"best_params": cv.best_params_, "best_score": cv.best_score_}, cv

In [91]:
#MAIN OUTPUT
models = []

for i,df in enumerate(dfs):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    convert_type(df)
    df = engineer_features(df)
    df = drop_features_and_na(df)
    
    X_train, X_test, y_train, y_test = split_data(df)
    
    model = create_pipeline()
    
    evaluation, cv = evaluate(model)
    df = df.drop(columns='Close')
    
    print("Best parameter:", evaluation['best_params']['lr__n_jobs'])
    print("Best score:", evaluation['best_score'])
    
    models.append({'model':cv,'X_test': X_test,'y_test': y_test})
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()
    
    # print_plots(df, stocks[i]) TODO: must implement merged stocks for good plotting visuals

Best parameter: 10
Best score: 0.999675972749141
AAPL:
         High        Low       Open     Volume       Date  Price Change  \
15  77.281827  76.634574  76.941293  101832400 2020-01-22      0.272911   
16  77.177978  76.233660  76.781900  104472000 2020-01-23      0.369514   
17  78.088493  76.685301  77.344635  146537600 2020-01-24     -0.222183   
18  75.296586  73.632564  74.883600  161940000 2020-01-27     -2.260574   
19  76.897829  75.398033  75.497055  162234000 2020-01-28      2.110832   

    Average Gain  Average Loss        RS        RSI  Short EMA   Long EMA  \
15      0.508213      0.208909  2.432705  70.868454  75.349467  74.270921   
16      0.534607      0.208909  2.559047  71.902589  75.618515  74.480355   
17      0.534607      0.174406  3.065305  75.401601  75.811990  74.657817   
18      0.493722      0.335875  1.469955  59.513426  75.627918  74.654684   
19      0.644495      0.311552  2.068662  67.412506  75.796909  74.808142   

        MACD  Signal Line  MACD

In [92]:
for i in range(len(models)):
    model_ = models[i]['model']
    X_test_ = models[i]['X_test']
    y_test_ = models[i]['y_test']
    
    y_pred_ = model_.predict(X_test_)
    
    print("MSE:", mean_squared_error(y_true=y_test_, y_pred=y_pred_))
    print("R2 Score:", r2_score(y_true=y_test_, y_pred=y_pred_))

    print(f"Predicted values for {stocks[i]}:")
    print(y_pred_[0:10])

    print(f"Correct values for {stocks[i]}:")
    print(y_test_[0:10])
    
    joblib.dump(model,f"models/{stocks[i]}.pkl")
    
    if joblib.load(f"models/{stocks[i]}.pkl"):
        print(f"{stocks[i]} model saved!")

MSE: 0.6632406663693234
R2 Score: 0.9996446759313484
Predicted values for AAPL:
[133.85924644 226.10597392 229.25878057 190.12899965 173.25178226
 151.84381928 141.87655082 213.65598819 206.69585044 192.09898036]
Correct values for AAPL:
378     133.879715
1280    227.203430
1234    229.056870
994     191.558197
493     172.296005
687     151.331161
708     141.161743
1407    213.008255
1354    206.625504
1003    191.974686
Name: Close, dtype: float64
AAPL model saved!
MSE: 0.7475446426048131
R2 Score: 0.9994624253464022
Predicted values for AMZN:
[172.70814521 236.81359192 198.19323473 144.5461563  169.73795238
 119.68284569 115.99680622 221.57695455 204.5917905  153.27894222]
Correct values for AMZN:
378     172.007996
1280    237.419998
1234    197.119995
994     145.889999
493     169.567505
687     118.540001
708     115.250000
1407    222.309998
1354    204.070007
1003    153.419998
Name: Close, dtype: float64
AMZN model saved!
MSE: 1.9503017993638658
R2 Score: 0.9997784619786328